In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

In [ ]:
CITY="DD"

In [ ]:

weather_filename_DD = f"../data/weather/df_Dresden_weather_hourly 2025-03-28_20-51-37.pkl"
weather_filename_FB = f"../data/weather/df_Freiburg_weather_hourly 2025-03-28_20-51-37.pkl"
weather_df_DD = pd.read_pickle(weather_filename_DD)
weather_df_FB = pd.read_pickle(weather_filename_FB)
help_dict_weather = {'DD': weather_df_DD, 'FB': weather_df_FB}

In [ ]:
df_weather = help_dict_weather[CITY]
df_weather.index.name = "datetime_hour"

In [ ]:
datetime = "2025-03-19_10-47-56"
filename_DD = f"../data/nextbike/hourly_demand_supply_Dresden {datetime}.csv"
filename_FB = f"../data/nextbike/hourly_demand_supply_Freiburg {datetime}.csv"
df_trips_DD = pd.read_csv(filename_DD, index_col=None, parse_dates=["datetime_hour"])
df_trips_FB = pd.read_csv(filename_FB, index_col=None, parse_dates=["datetime_hour"])


help_dict = {'DD': df_trips_DD, 'FB': df_trips_FB}

In [ ]:
weather_df_FB.index.name = "datetime_hour"

In [ ]:
df_trips = help_dict[CITY]

In [ ]:
df_houly_trips= df_trips.drop(columns=["hex_id", "return_count"]).groupby("datetime_hour").agg("sum")

In [ ]:
# df_houly_trips

In [ ]:
df = df_houly_trips.copy()

In [ ]:
len(df)

In [ ]:
df_weather.loc[df.index]

In [ ]:
df = pd.concat([df, df_weather.loc[df.index]], axis=1)

In [ ]:
df.rename(columns={"rent_count": "Bike trips"}, inplace=True)

In [ ]:
df_total =  df.reset_index().resample("h", on="datetime_hour").agg({"Temperature": "mean", "Humidity": "mean", "Wind": "mean", "Precipitation": "sum", "Bike trips": "sum"})
df_total.dropna(subset="Temperature", inplace=True)

df_total.Temperature = df_total.Temperature.astype(int)
df_total.Humidity = df_total.Humidity.astype(int)


In [ ]:
df_total['Rain']= df_total['Precipitation'].apply(lambda x: True if x > 0 else False)

In [ ]:
cols =  [col for col in df_total.columns if col != "Bike trips"]+["Bike trips"] 


df_total = df_total[cols]

In [ ]:
df_total_d =  df_total.reset_index().resample("d", on="datetime_hour").agg({"Temperature": "mean", "Humidity": "mean", "Wind": "mean", "Precipitation": "sum", "Bike trips": "sum"})

In [ ]:
df_total_d

In [ ]:
px.parallel_coordinates(df_total_d, color="Bike trips", width=800, height=600, ).show()

In [ ]:
df_total_d

In [ ]:
df_total_d.reset_index(inplace=True)

In [ ]:
import plotly.graph_objects as go

In [ ]:
df_total_d.dropna(subset="Temperature", inplace=True)

In [ ]:
df_total_d["Date"] =df_total_d.datetime_hour.dt.date.astype(str)

In [ ]:
df_total_d

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_total_d.Date, y=df_total_d["Temperature"], mode='lines', name='Temperature'))
fig.add_trace(go.Scatter(x=df_total_d.Date, y=df_total_d["Bike trips"], mode='lines', name='Bike trips', yaxis='y2'))
fig.update_layout(
    yaxis=dict(title="Temperature"),
    yaxis2=dict(title="Bike Trips", overlaying='y', side='right'),
    xaxis=dict(title="Date", type='category'),  # Treat x-axis as categories to skip missing dates
    legend=dict(x=0.8, y=1),  # Adjust legend position
    width=800, height=600
)
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_total_d.Date, y=df_total_d["Temperature"], mode='lines', name='Temperature'))
fig.add_trace(go.Scatter(x=df_total_d.Date, y=df_total_d["Bike trips"], mode='lines', name='Bike trips', yaxis='y2'))
fig.update_layout(
    yaxis=dict(title="Temperature"),
    yaxis2=dict(title="Bike Trips", overlaying='y', side='right'),
    xaxis=dict(title="Date", type='category'),  # Treat x-axis as categories to skip missing dates
    legend=dict(x=0.8, y=1),  # Adjust legend position
    width=800, height=600
)
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_total_d.Date, y=df_total_d["Humidity"], mode='lines', name='Humidity'))
fig.add_trace(go.Scatter(x=df_total_d.Date, y=df_total_d["Bike trips"], mode='lines', name='Bike trips', yaxis='y2'))
fig.update_layout(
    yaxis=dict(title="Humidity"),
    yaxis2=dict(title="Bike Trips", overlaying='y', side='right'),
    xaxis=dict(title="Date", type='category'),  # Treat x-axis as categories to skip missing dates
    legend=dict(x=0.8, y=1),  # Adjust legend position
    width=800, height=600
)
fig.show()


In [ ]:
df_total

In [ ]:
fig = go.Figure()
n_days = 20
fig.add_trace(go.Scatter(x=df_total.index[:24*n_days], y=df_total["Temperature"].iloc[:24*n_days], mode='lines', name='Temperature'))
fig.add_trace(go.Scatter(x=df_total.index[:24*n_days], y=df_total["Bike trips"].iloc[:24*n_days], mode='lines', name='Bike trips', yaxis='y2'))
fig.update_layout(
    yaxis=dict(title="Temperature"),
    yaxis2=dict(title="Bike Trips", overlaying='y', side='right'),
    xaxis=dict(title="Date"),  # Treat x-axis as categories to skip missing dates
    legend=dict(x=1.1, y=1),  # Adjust legend position
    width=800, height=600
)
fig.show()
